# The Solar System
__*Diego De Gusem*__
<br>
__*Very special thanks to Ian Lateur*__

## 0 Table of contents
* [1 Used formulas](#formulas)
    * [1.1 Gravity](#gravity)
    * [1.2 Euler integration](#euler)
* [2 Used data](#data)
    * [2.1 NASA](#nasa)
* [3 The Solar System](#system)
    * [3.1 The gravitational constant](#constant)
    * [3.2 The objects of our solar system](#objects)
    * [3.3 The Solar System](#solar)
    * [3.4 The date](#date)
    * [3.5 Animations](#animations)
        * [3.5.1 Still animation](#still)
        * [3.5.2 Zoom](#zoom)
* [4 Your System](#your)
* [5 Conversion](#convertion)
* [6 Extra](#extra)

## 1 Used formulas <a class="anchor" id="formulas"></a>

### 1.1 Gravity <a class="anchor" id="gravity"></a>
We use the following formula to calculate the acceleration (change in velocity) of our planets and sun
\begin{align}
    \vec{a} = -\frac{GM}{|\vec{r}|^2}\frac{\vec{r}}{|\vec{r}|}
\end{align}
### 1.2 Euler integration <a class="anchor" id="euler"></a>
To calculate our position after a small amount of time ($dt$), we use
\begin{align}
    \vec{r_{new}} &= \vec{r_{old}} + \vec{v_{old}} \cdot dt\\
    \vec{v_{new}} &= \vec{v_{old}} + \vec{a_{new}} \cdot dt
\end{align}

## 2 Used Data <a class="anchor" id="data"></a>

### 2.1 NASA <a class="anchor" id="nasa"></a>
The data we used for our initial conditions was *2020-Oct-28 00:00:00.0000*. We could find these values in a NASA library. You can check out this library by clicking on the following link:
https://ssd.jpl.nasa.gov/horizons.cgi#results. You can also find our falues [here](#extra).

## 3 The System <a class="anchor" id="system"></a>

In [ ]:
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sympy import *
from mpl_toolkits.mplot3d import *
import mpl_toolkits.mplot3d.axes3d as p3
from datetime import timedelta
import datetime
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

from IPython.display import HTML
from matplotlib import animation

### 3.1 The gravitational constant <a class="anchor" id="constant"></a>
To make things easier for ourselves we adjusted the gravitational constant so its units would be in $\frac{AU}{M_{sun}\cdot days^2}$.

In [ ]:
G = 6.67408e-11 * ((1.989e30*86400**2)/149597870700**3)

### 3.2 The objects of our solar system <a class="anchor" id="objects"></a>
We created a class for our objects. This class has everything we need from our objects in our solar system. Every object gets a name, mass, radius, position, velocity and color. This class can also calculate the distance between two objects and the acceleration of one object because of another object.

In [ ]:
class Object:
    def __init__(self, name, mass, rad, p, v, color):
        self.name = name
        self.mass = mass
        self.rad = rad #Radius
        self.p = np.array(p) #Position
        self.v = np.array(v) #Velocity
        self.color = color
        self.x, self.y, self.z = [self.p[0]], [self.p[1]], [self.p[2]]
    
    def distance(self, other):
        return self.p - other.p
    
    def acceleration(self, other):
        r = self.distance(other)
        return -G * other.mass /norm(r)**2 * r/norm(r)

### 3.3 The Solar System <a class="anchor" id="solar"></a>
We also created a class for our solar system. To use our solar system we first need all of our planets and our sun. This class can calculate the total acceleration of one object because of all the other objects in our system. It can also simulate the behaviour of our system. 

In [ ]:
class SolarSystem:
    def __init__(self, planets):
        self.planets = planets
    
    def total_acceleration(self):
        forces = []
        for planet in self.planets:
            a = [0, 0, 0]
            for i in range(len(self.planets)):
                if self.planets[i] != planet:
                    a += planet.acceleration(self.planets[i])
            forces.append(a)
        return forces
    
    def change(self, dt):
        v_new, p_new = [], []
        forces = self.total_acceleration()
        for i, planet in enumerate(self.planets):
            p_new.append(planet.p + planet.v * dt)
            v_new.append(planet.v + forces[i] * dt)
        for i, planet in enumerate(self.planets):
            planet.p = p_new[i]
            planet.v = v_new[i]
            planet.x.append(planet.p[0])
            planet.y.append(planet.p[1])
            planet.z.append(planet.p[2])

### 3.4 The date <a class="anchor" id="date"></a>
Now we have our planets and our solar system. But we also want to know at what time we will see a given configuration.

To calculate the date we first need a way to calculate the angle between the earth, sun and the vernal equinox, a.k.a. the ecliptic length ($\lambda$).

We know that $\lambda = \arctan\left(\frac{y}{x}\right)$ in the interval $[0, 2\pi]$. So we need to calculate the position of our earth relative to our sun to find our $x$ and $y$ components. Once we have this we just need a way to calculate the $\arctan$. When this is done we can easily calculate the current date.

In [ ]:
def arct(x, y):
    if y>=0 and x>0:
        return np.arctan(y/x)
 
    elif x<0:
        return np.pi + np.arctan(y/x)

    elif x>0 and y<0:
        return 2*np.pi + np.arctan(y/x)

    elif y>0 and x==0:
        return np.pi/2
    
    elif y<0 and x==0:
        return -np.pi/2

In [ ]:
def date(pos1, pos2, year):
    autumn = datetime.datetime(year, 9, 23)
    
    dist = pos1 - pos2
    lam = arct(dist[0], dist[1])
    now = autumn + timedelta(days=float(365.242199 * (lam)/(2*np.pi)))
    return now

### 3.5 Animations <a class="anchor" id="animations"></a>
Now we will show you the different type of animations you can use

#### 3.5.1 Still animation<a class="anchor" id="still"></a>
Our first type of animation will simulate the position of your objects.

In [ ]:
def still(i):
    global dates, dat, year, max_d
    for lnum, line in enumerate(lines):
        planet = planets[lnum]
        line.set_data(planet.x[:i], planet.y[:i])
        line.set_3d_properties(planet.z[:i])
    
    scat._offsets3d = ([planet.x[i] for planet in planets], [planet.y[i] for planet in planets], [planet.z[i] for planet in planets])
    
    if dates.value:
        if (dat - date(np.array([Earth.x[i], Earth.y[i], Earth.z[i]]), np.array([Sun.x[i], Sun.y[i], Sun.z[i]]), year)).days > 300:
            year.value += 1
        dat = date(np.array([Earth.x[i], Earth.y[i], Earth.z[i]]), np.array([Sun.x[i], Sun.y[i], Sun.z[i]]), year)

        colors = [planet.color for planet in planets]
        texts = [planet.name for planet in planets]
        patches = [ax.plot([],[],[], marker=".", ms=10, ls="", mec=None, color=colors[i], label="{:s}".format(texts[i]) )[0]  for i in range(len(texts))]
        l = ax.legend(handles=patches, loc='lower left', title='Date\n {}/{}/{}'.format(dat.year, dat.month, dat.day))
        plt.setp(l.get_title(), multialignment='center')
    
    else:
        colors = [planet.color for planet in planets]
        texts = [planet.name for planet in planets]
        patches = [ax.plot([],[],[], marker=".", ms=10, ls="", mec=None, color=colors[i], label="{:s}".format(texts[i]) )[0]  for i in range(len(texts))]
        l = ax.legend(handles=patches, loc='lower left')

    ax.set(xlim=(-max_d, max_d), ylim=(-max_d, max_d), zlim=(-max_d, max_d))
    
    return lines

#### 3.5.2 Zoom out <a class="anchor" id="zoom"></a>
You can use this animation to zoom out.

In [ ]:
def zoom(i):
    global dates, dat, year, zoom
    for lnum, line in enumerate(lines):
        planet = planets[lnum]
        line.set_data(planet.x[:i], planet.y[:i])
        line.set_3d_properties(planet.z[:i])
    
    scat._offsets3d = ([planet.x[i] for planet in planets], [planet.y[i] for planet in planets], [planet.z[i] for planet in planets])
    
    if dates.value:
        if (dat - date(np.array([Earth.x[i], Earth.y[i], Earth.z[i]]), np.array([Sun.x[i], Sun.y[i], Sun.z[i]]), year)).days > 300:
            year += 1
        dat = date(np.array([Earth.x[i], Earth.y[i], Earth.z[i]]), np.array([Sun.x[i], Sun.y[i], Sun.z[i]]), year)
    
        colors = [planet.color for planet in planets]
        texts = [planet.name for planet in planets]
        patches = [ax.plot([],[],[], marker=".", ms=10, ls="", mec=None, color=colors[i], label="{:s}".format(texts[i]) )[0]  for i in range(len(texts))]
        l = ax.legend(handles=patches, loc='lower left', title='Date\n {}/{}/{}'.format(dat.year, dat.month, dat.day))
        plt.setp(l.get_title(), multialignment='center')
    
    else:
        colors = [planet.color for planet in planets]
        texts = [planet.name for planet in planets]
        patches = [ax.plot([],[],[], marker=".", ms=10, ls="", mec=None, color=colors[i], label="{:s}".format(texts[i]) )[0]  for i in range(len(texts))]
        l = ax.legend(handles=patches, loc='lower left')
    
    ax.set(xlim=Zoom[i], ylim=Zoom[i], zlim=Zoom[i])
    return lines

## 4 Your System <a class="anchor" id="your"></a>
Now that we've explained our code to you, it's your turn. You can just copy our code and adjust it __or__ you can use the following code by filling in the boxes!

In [ ]:
planets = []
name = widgets.Text(value='', description='Name')
mass = widgets.Text(value='', description='Mass')
rad = widgets.Text(value='', description='Radius')
x, y, z = widgets.Text(value='', description='x position'), widgets.Text(value='', description='y position'), widgets.Text(value='', description='z position')
vx, vy, vz = widgets.Text(value='', description='x velocity'), widgets.Text(value='', description='y velocity'), widgets.Text(value='', description='z velocity')
color = widgets.Text(value='', description='Color')

years = widgets.FloatSlider(value=0.1,min=0.1,max=5.0,step=0.1, description='years to simulate')
dates = widgets.Checkbox(description='Sun and Earth')
starting_year = widgets.Text(value='2020', description='Starting year')

fps = widgets.Text(value='', description='Fps')
frames = widgets.Text(value='', description='Amount of frames')
function = widgets.Dropdown(options=['still', 'zoom'], value='still', description='Type of animation:')
sa = widgets.Text(value='', description='Name of the animation')

button = widgets.Button(description='Add Object')

@button.on_click
def app(b):
    planets.append(Object(str(name.value), float(mass.value), float(rad.value), [float(x.value), float(y.value), float(z.value)], [float(vx.value), float(vy.value), float(vz.value)], str(color.value)))

info = widgets.VBox([name, mass, rad, x, y, z, vx, vy, vz, color, button])
time = widgets.VBox([years, dates, starting_year])
anim = widgets.VBox([function, fps, frames, sa])

In [ ]:
children = [info, time, anim]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'Objects info')
tab.set_title(1, 'Time info')
tab.set_title(2, 'Animation info')

tab

In [ ]:
ss = SolarSystem(planets)

max_d, min_d = norm(planets[0].p), norm(planets[0].p)
for i in planets:
    if norm(i.p) > max_d:
        max_d = norm(i.p)
    if norm(i.p) < min_d:
        min_d = norm(i.p)

In [ ]:
dt = 0.01
for _ in np.arange(float(years.value)*365.242199/dt):
    ss.change(dt)

In [ ]:
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.grid(False)
fig.set_facecolor('black')
ax.set_facecolor('black')
ax.w_xaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.w_yaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.w_zaxis.set_pane_color((0.0, 0.0, 0.0, 0.0))
ax.w_xaxis.line.set_color('white')
ax.w_yaxis.line.set_color('white')
ax.w_zaxis.line.set_color('white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.zaxis.label.set_color('white')
ax.tick_params(colors='white')
ax.grid(True)
ax.set(xlabel='x (AU)', ylabel='y (AU)', zlabel='z (AU)')

lines = []

for planet in planets:
    lobj = ax.plot([planet.x[0]],[planet.y[0]],[planet.z[0]], color=planet.color)[0]
    lines.append(lobj)

scat = ax.scatter([planet.x[0] for planet in planets], [planet.y[0] for planet in planets], [planet.z[0] for planet in planets], marker='o', color=[planet.color for planet in planets], alpha=1, label=planet.name)

year = float(starting_year.value)

if dates.value:
    dat = date(np.array([Earth.x[0], Earth.y[0], Earth.z[0]]), np.array([Sun.x[0], Sun.z[0], Sun.z[0]]), year)
else:
    dat = None

Zoom = [(- np.exp(i), np.exp(i)) for i in np.linspace(np.log(min_d), np.log(max_d), len(planets[0].x))]

In [ ]:
if str(function.value) == "still":
    ani = animation.FuncAnimation(fig, still, frames=range(0, len(planets[0].x), int(len(planets[0].x)/float(frames.value))), interval=1000/float(fps.value), blit=True)
else:
    ani = animation.FuncAnimation(fig, zoom, frames=range(0, len(planets[0].x), int(len(planets[0].x)/float(frames.value))), interval=1000/(fps.value), blit=True)

ani.save(sa.value, dpi=200)
print("DONE")

## 5 Conversion <a class="anchor" id="convertion"></a>
If you do not know how to convert your values, no problem, we are here to help!

In [ ]:
M = widgets.Text(value='', description='Mass in kg')
d = widgets.Text(value='', description='Distance in m')
v = widgets.Text(value='', description='Velocity in m/s')

Mbutton = widgets.Button(description='kg to Solar mass')
dbutton = widgets.Button(description='m to AU')
vbutton = widgets.Button(description='m/s to AU/day')

@Mbutton.on_click
def app(b):
    print(float(M.value)*1.9891e30, 'Solar mass')

@dbutton.on_click
def app(b):
    print(float(d.value)*6.68458712e-12, 'AU')

@vbutton.on_click
def app(b):
    print(float(v.value)*5.77548327e-7, 'AU/day')
    
conv = widgets.VBox([M, Mbutton, d, dbutton, v, vbutton])

In [ ]:
conv

## 6 Extra <a class="anchor" id="extra"></a>